In [1]:
import copy
import re
import pandas as pd
import numpy as np
from experiments import ExperimentModule
from experiments import (
    save_plots, 
    write_row, 
    write_opcounts,
    save_circuit,
    load_qasm_circuit,
    create_dir,
    get_random_state
)
import matplotlib.pyplot as plt
from itertools import product

%matplotlib inline

In [2]:
results_dir = "results/runs_shell_w_entanglement/random_states"
run_range = list(range(1, 11))
#qubit_range = [3, 4, 5, 6, 7, 8]
qubit_range = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#qubit_range = [3, 4, 5, 6, 7, 8, 9, 10]

# EPS values are mapped separately
#eps_values = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
eps_values = [0.5, 0.1, 0.05, 0.01]

### Random State type

In [3]:
state_type = "random_state"

In [4]:
#[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
original_results_dict = {
    "num_qubits": [],
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}

modified_results_dict = copy.deepcopy(original_results_dict)

for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
    if not num_qubits in original_results_dict["num_qubits"]: 
        original_results_dict["num_qubits"] += [num_qubits]
        modified_results_dict["num_qubits"] += [num_qubits]

    original_results_dict[eps] += [[]]
    modified_results_dict[eps] += [[]]

    for run_idx in run_range:
        fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
        original_results_dict[eps][-1] += [data_original.round(4)]

        fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
        modified_results_dict[eps][-1]+= [data_modified.round(4)]

In [5]:
default_dict = {
    "num_qubits": qubit_range,
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}
mean_original_results_dict = copy.deepcopy(default_dict)
mean_modified_results_dict = copy.deepcopy(default_dict)

stds_original_results_dict = copy.deepcopy(default_dict)
stds_modified_results_dict = copy.deepcopy(default_dict)

for eps in eps_values:
    mean_original_results_dict[eps] = np.mean(original_results_dict[eps], axis=1)
    stds_original_results_dict[eps] = np.std(original_results_dict[eps], axis=1).round(2)

    mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
    stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)

Original fidloss

In [6]:
df_mean_original = pd.DataFrame(mean_original_results_dict)
df_mean_original = df_mean_original.set_index("num_qubits")
display(df_mean_original)

df_stds_original = pd.DataFrame(stds_original_results_dict)
df_stds_original = df_stds_original.set_index("num_qubits")
df_stds_original

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.04930,0.04930,0.00088,0.00145
4,0.15660,0.15660,0.10955,0.00208
5,0.19660,0.19660,0.53126,0.12929
6,0.22060,0.22060,0.20610,0.17372
7,0.23790,0.23790,0.23220,0.21141
8,0.23392,0.23390,0.74547,0.22380
9,0.25440,0.25440,0.81773,0.74473
10,0.75682,0.74582,0.59554,0.90523
11,0.49663,0.53897,0.65102,0.69986


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.06,0.00
6,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.05,0.00
9,0.00,0.00,0.03,0.04
10,0.04,0.03,0.08,0.02
11,0.07,0.16,0.02,0.05


In [7]:
print(df_mean_original.to_latex(float_format="%.4f"))

\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.0493 & 0.0493 & 0.0009 & 0.0015 \\
4 & 0.1566 & 0.1566 & 0.1095 & 0.0021 \\
5 & 0.1966 & 0.1966 & 0.5313 & 0.1293 \\
6 & 0.2206 & 0.2206 & 0.2061 & 0.1737 \\
7 & 0.2379 & 0.2379 & 0.2322 & 0.2114 \\
8 & 0.2339 & 0.2339 & 0.7455 & 0.2238 \\
9 & 0.2544 & 0.2544 & 0.8177 & 0.7447 \\
10 & 0.7568 & 0.7458 & 0.5955 & 0.9052 \\
11 & 0.4966 & 0.5390 & 0.6510 & 0.6999 \\
12 & 0.9446 & 0.9487 & 0.6807 & 0.9180 \\
13 & 0.7394 & 0.7446 & 0.6686 & 0.9494 \\
14 & 0.9870 & 0.9839 & 0.9077 & 0.9716 \\
15 & 0.9318 & 0.9120 & 0.9557 & 0.9952 \\
\bottomrule
\end{tabular}



Modified fidloss

In [8]:
df_mean_modified = pd.DataFrame(mean_modified_results_dict)
df_mean_modified = df_mean_modified.set_index("num_qubits")
display(df_mean_modified)

df_stds_modified = pd.DataFrame(stds_modified_results_dict)
df_stds_modified = df_stds_modified.set_index("num_qubits")
df_stds_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.00540,0.00393
4,0.17420,0.17420,0.12373,0.00090
5,0.18270,0.18270,0.71420,0.13962
6,0.21140,0.21140,0.56862,0.71236
7,0.24200,0.24200,0.60720,0.21420
8,0.23310,0.23310,0.22610,0.91826
9,0.25090,0.25090,0.92590,0.64514
10,0.88302,0.89487,0.49291,0.99206
11,0.89121,0.89567,0.95305,0.24954


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.01,0.01
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00
6,0.00,0.00,0.08,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.02
9,0.00,0.00,0.02,0.06
10,0.02,0.02,0.09,0.00
11,0.02,0.02,0.01,0.00


In [9]:
print(df_mean_modified.to_latex(float_format="%.4f"))

\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.0595 & 0.0595 & 0.0054 & 0.0039 \\
4 & 0.1742 & 0.1742 & 0.1237 & 0.0009 \\
5 & 0.1827 & 0.1827 & 0.7142 & 0.1396 \\
6 & 0.2114 & 0.2114 & 0.5686 & 0.7124 \\
7 & 0.2420 & 0.2420 & 0.6072 & 0.2142 \\
8 & 0.2331 & 0.2331 & 0.2261 & 0.9183 \\
9 & 0.2509 & 0.2509 & 0.9259 & 0.6451 \\
10 & 0.8830 & 0.8949 & 0.4929 & 0.9921 \\
11 & 0.8912 & 0.8957 & 0.9531 & 0.2495 \\
12 & 0.8590 & 0.8646 & 0.9316 & 0.9832 \\
13 & 0.9488 & 0.9470 & 0.9753 & 0.9770 \\
14 & 0.9953 & 0.9937 & 0.8216 & 0.9932 \\
15 & 0.9858 & 0.9853 & 0.9997 & 0.9585 \\
\bottomrule
\end{tabular}



In [10]:
df_mean_original < df_mean_modified


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,True,True,True,True
4,True,True,True,False
5,False,False,True,True
6,False,False,True,True
7,True,True,True,True
8,False,False,False,True
9,False,False,True,False
10,True,True,False,True
11,True,True,True,False


### Random Sparse States

In [11]:
state_type = "random_sparse_state"
density = 0.3

In [12]:
#[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
original_results_dict = {
    "num_qubits": [],
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}

modified_results_dict = copy.deepcopy(original_results_dict)

for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
    if not num_qubits in original_results_dict["num_qubits"]: 
        original_results_dict["num_qubits"] += [num_qubits]
        modified_results_dict["num_qubits"] += [num_qubits]

    original_results_dict[eps] += [[]]
    modified_results_dict[eps] += [[]]

    for run_idx in run_range:
        fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/density_{density}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
        original_results_dict[eps][-1] += [data_original.round(4)]

        fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/density_{density}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
        modified_results_dict[eps][-1]+= [data_modified.round(4)]

In [13]:
default_dict = {
    "num_qubits": qubit_range,
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}
mean_original_results_dict = copy.deepcopy(default_dict)
mean_modified_results_dict = copy.deepcopy(default_dict)

stds_original_results_dict = copy.deepcopy(default_dict)
stds_modified_results_dict = copy.deepcopy(default_dict)

for eps in eps_values:
    mean_original_results_dict[eps] = np.mean(original_results_dict[eps], axis=1)
    stds_original_results_dict[eps] = np.std(original_results_dict[eps], axis=1).round(2)

    mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
    stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)

In [14]:
df_mean_original = pd.DataFrame(mean_original_results_dict)
df_mean_original = df_mean_original.set_index("num_qubits")
display(df_mean_original)
display(df_mean_modified)

df_stds_original = pd.DataFrame(stds_original_results_dict)
df_stds_original = df_stds_original.set_index("num_qubits")
#df_stds_original

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.30224,0.34575,0.00000,0.00000
4,0.79322,0.78398,0.91706,0.00000
5,0.59391,0.59390,0.64485,0.27443
6,0.64480,0.66716,0.62177,0.50721
7,0.72480,0.72480,0.66490,0.62675
8,0.77136,0.75500,0.86537,0.70798
9,0.75820,0.77832,0.92437,0.75093
10,0.79122,0.78622,0.77561,0.84350
11,0.85540,0.84438,0.82956,0.88915


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.00540,0.00393
4,0.17420,0.17420,0.12373,0.00090
5,0.18270,0.18270,0.71420,0.13962
6,0.21140,0.21140,0.56862,0.71236
7,0.24200,0.24200,0.60720,0.21420
8,0.23310,0.23310,0.22610,0.91826
9,0.25090,0.25090,0.92590,0.64514
10,0.88302,0.89487,0.49291,0.99206
11,0.89121,0.89567,0.95305,0.24954


In [15]:
display(df_stds_original)
display(df_stds_modified)

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.12,0.17,0.00,0.00
4,0.11,0.05,0.07,0.00
5,0.00,0.00,0.04,0.03
6,0.00,0.07,0.01,0.00
7,0.00,0.00,0.00,0.00
8,0.05,0.00,0.02,0.00
9,0.00,0.06,0.01,0.00
10,0.02,0.02,0.01,0.01
11,0.02,0.01,0.06,0.02


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.01,0.01
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00
6,0.00,0.00,0.08,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.02
9,0.00,0.00,0.02,0.06
10,0.02,0.02,0.09,0.00
11,0.02,0.02,0.01,0.00


In [16]:
print("--"*100)
print("Original Mean")
print("--"*100)
print(df_mean_original.to_latex(float_format="%.4f"))

print("--"*100)
print("Modified Mean")
print("--"*100)
print(df_mean_modified.to_latex(float_format="%.4f"))

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Original Mean
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.3022 & 0.3458 & 0.0000 & 0.0000 \\
4 & 0.7932 & 0.7840 & 0.9171 & 0.0000 \\
5 & 0.5939 & 0.5939 & 0.6449 & 0.2744 \\
6 & 0.6448 & 0.6672 & 0.6218 & 0.5072 \\
7 & 0.7248 & 0.7248 & 0.6649 & 0.6268 \\
8 & 0.7714 & 0.7550 & 0.8654 & 0.7080 \\
9 & 0.7582 & 0.7783 & 0.9244 & 0.7509 \\
10 & 0.7912 & 0.7862 & 0.7756 & 0.8435 \\
11 & 0.8554 & 0.8444 & 0.8296 & 0.8892 \\
12 & 0.9681 & 0.9641 & 0.8584 & 0.9690 \\
13 & 0.9163 & 0.9162 & 0.9095 & 0.9821 \\
14 & 0.9886 & 

In [17]:
df_mean_original < df_mean_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,False,False,True,True
4,False,False,False,True
5,False,False,True,False
6,False,False,False,True
7,False,False,False,False
8,False,False,False,True
9,False,False,True,False
10,True,True,False,True
11,True,True,True,False
